In [1]:
import torch
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

class FederatedClient:
    def __init__(self, model, data_loader, lr=0.01):
        self.model = model
        self.data_loader = data_loader
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = torch.nn.MSELoss()
    
    def train(self, epochs=1):
        """ Train the model on local data """
        self.model.train()
        for epoch in range(epochs):
            for batch in self.data_loader:
                self.optimizer.zero_grad()
                out = self.model(batch)
                loss = self.criterion(out, batch.edge_attr.view(-1, 1))
                loss.backward()
                self.optimizer.step()
            print(f'Client Epoch {epoch + 1}, Loss: {loss.item()}')

    def evaluate(self, data_loader):
        """ Evaluate the model on a given data loader """
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in data_loader:
                out = self.model(batch)
                loss = self.criterion(out, batch.edge_attr.view(-1, 1))
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def get_model(self):
        return self.model
    
    def get_gradients(self):
        """ Get the gradients of the model parameters """
        gradients = {}
        for name, param in self.model.named_parameters():
            if param.grad is not None:
                gradients[name] = param.grad.mean().item()
        return gradients


In [2]:
import torch
from collections import OrderedDict

class FederatedServer:
    def __init__(self, model, num_clients, lr=0.01):
        self.model = model
        self.num_clients = num_clients
        self.lr = lr
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def aggregate(self, client_models):
        """ Aggregate model parameters from clients """
        state_dicts = [client_model.state_dict() for client_model in client_models]
        average_state_dict = OrderedDict()

        for key in state_dicts[0].keys():
            average_state_dict[key] = torch.mean(
                torch.stack([state_dict[key].float() for state_dict in state_dicts]), dim=0
            )
        
        self.model.load_state_dict(average_state_dict)
    
    def evaluate(self, test_loader):
        """ Evaluate the global model """
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in test_loader:
                out = self.model(batch)
                loss = F.mse_loss(out, batch.edge_attr.view(-1, 1))
                total_loss += loss.item()
        return total_loss / len(test_loader)
    
    def get_gradients(self):
        """ Get the gradients of the global model parameters """
        gradients = {}
        for name, param in self.model.named_parameters():
            if param.grad is not None:
                gradients[name] = param.grad.mean().item()
        return gradients


In [4]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels * 2, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        edge_pred = self.fc(torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=1))
        return edge_pred.squeeze()

# Define the Federated Client class
class FederatedClient:
    def __init__(self, model, data_loader, lr=0.01):
        self.model = model
        self.data_loader = data_loader
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = torch.nn.MSELoss()
    
    def train(self, epochs=1):
        """ Train the model on local data """
        self.model.train()
        for epoch in range(epochs):
            for batch in self.data_loader:
                self.optimizer.zero_grad()
                out = self.model(batch)
                loss = self.criterion(out, batch.edge_attr.view(-1, 1))
                loss.backward()
                self.optimizer.step()
            print(f'Client Epoch {epoch + 1}, Loss: {loss.item()}')

    def evaluate(self, data_loader):
        """ Evaluate the model on a given data loader """
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in data_loader:
                out = self.model(batch)
                loss = self.criterion(out, batch.edge_attr.view(-1, 1))
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def get_model(self):
        return self.model
    
    def get_gradients(self):
        """ Get the gradients of the model parameters """
        gradients = {}
        for name, param in self.model.named_parameters():
            if param.grad is not None:
                gradients[name] = param.grad.mean().item()
        return gradients

# Define the Federated Server class
class FederatedServer:
    def __init__(self, model, num_clients, lr=0.01):
        self.model = model
        self.num_clients = num_clients
        self.lr = lr
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def aggregate(self, client_models):
        """ Aggregate model parameters from clients """
        state_dicts = [client_model.state_dict() for client_model in client_models]
        average_state_dict = {key: torch.mean(torch.stack([state_dict[key].float() for state_dict in state_dicts]), dim=0) for key in state_dicts[0]}
        self.model.load_state_dict(average_state_dict)
    
    def evaluate(self, test_loader):
        """ Evaluate the global model """
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in test_loader:
                out = self.model(batch)
                loss = F.mse_loss(out, batch.edge_attr.view(-1, 1))
                total_loss += loss.item()
        return total_loss / len(test_loader)
    
    def get_gradients(self):
        """ Get the gradients of the global model parameters """
        gradients = {}
        for name, param in self.model.named_parameters():
            if param.grad is not None:
                gradients[name] = param.grad.mean().item()
        return gradients

# Load and preprocess data
def load_data():
    url = "ratings_Electronics (1).csv"
    df = pd.read_csv(url)
    df.rename(columns={'AKM1MP6P0OYPR': 'userId', '0132793040': 'productId', '5.0': 'Rating', '1365811200': 'timestamp'}, inplace=True)
    df = df.head(5000)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    user_encoder = LabelEncoder()
    item_encoder = LabelEncoder()
    df['userId'] = user_encoder.fit_transform(df['userId'])
    df['productId'] = item_encoder.fit_transform(df['productId'])

    return df

def create_data_objects(df, split_ratio=0.2):
    train_df, test_df = train_test_split(df, test_size=split_ratio, random_state=42)
    train_df, val_df = train_test_split(train_df, test_size=split_ratio, random_state=42)

    edge_index = torch.tensor([train_df['userId'].values, train_df['productId'].values], dtype=torch.long)
    edge_attr = torch.tensor(train_df['Rating'].values, dtype=torch.float)
    num_users = df['userId'].nunique()
    num_items = df['productId'].nunique()
    num_nodes = num_users + num_items
    node_features = torch.eye(num_nodes)
    data = Data(edge_index=edge_index, edge_attr=edge_attr, x=node_features)
    
    train_loader = DataLoader([data], batch_size=1, shuffle=True)
    val_loader = DataLoader([data], batch_size=1, shuffle=False)
    
    return data, train_loader, val_loader

# Initialize models and federated setup
num_clients = 3
df = load_data()
client_data = [create_data_objects(df)[0] for _ in range(num_clients)]
client_train_loaders = [create_data_objects(df)[1] for _ in range(num_clients)]
client_val_loaders = [create_data_objects(df)[2] for _ in range(num_clients)]

# Initialize the global model and server
model = GCN(in_channels=client_data[0].x.size(1), hidden_channels=16, out_channels=1)
server = FederatedServer(model, num_clients)

# Federated learning rounds
for round in range(5):  # Number of federated learning rounds
    client_models = []
    
    # Train clients
    for i, client_loader in enumerate(client_train_loaders):
        client = FederatedClient(model, client_loader)
        client.train(epochs=1)  # Train each client for 1 epoch
        client_models.append(client.get_model())
        
        # Evaluate client model
        val_loss = client.evaluate(client_val_loaders[i])
        print(f'Client {i + 1} Validation Loss: {val_loss}')
        
        # Inspect gradients
        gradients = client.get_gradients()
        print(f'Client {i + 1} Gradients: {gradients}')
    
    # Aggregate client models
    server.aggregate(client_models)
    
    # Evaluate global model
    test_loader = create_data_objects(df)[2]  # Use test loader for evaluation
    global_val_loss = server.evaluate(test_loader)
    print(f'Round {round + 1} Global Validation Loss: {global_val_loss}')
    
    # Inspect server model gradients
    server_gradients = server.get_gradients()
    print(f'Server Gradients: {server_gradients}')
    
    print(f'Round {round + 1} completed')


C:\Users\XhaniD\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
C:\Users\XhaniD\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([3200, 1])) that is different to the input size (torch.Size([3200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Client Epoch 1, Loss: 19.113727569580078
Client 1 Validation Loss: 18.13140869140625
Client 1 Gradients: {'conv1.bias': 1.0709213018417358, 'conv1.lin.weight': 0.00020956707885488868, 'conv2.bias': -0.046430304646492004, 'conv2.lin.weight': -0.004444539546966553, 'fc.weight': -0.3210436701774597, 'fc.bias': -8.268858909606934}
Client Epoch 1, Loss: 18.13140869140625
Client 2 Validation Loss: 17.025196075439453
Client 2 Gradients: {'conv1.bias': -0.7820969223976135, 'conv1.lin.weight': -0.00015519207227043808, 'conv2.bias': -0.2402704656124115, 'conv2.lin.weight': -0.008603132329881191, 'fc.weight': -0.35288503766059875, 'fc.bias': -8.027332305908203}
Client Epoch 1, Loss: 17.025196075439453
Client 3 Validation Loss: 15.4755277633667
Client 3 Gradients: {'conv1.bias': -2.2816295623779297, 'conv1.lin.weight': -0.00044579882523976266, 'conv2.bias': -0.39783018827438354, 'conv2.lin.weight': -0.03395041450858116, 'fc.weight': -0.6436702013015747, 'fc.bias': -7.743130683898926}
Round 1 Globa

C:\Users\XhaniD\AppData\Local\Temp\ipykernel_12600\2777615108.py:89: UserWarning: Using a target size (torch.Size([3200, 1])) that is different to the input size (torch.Size([3200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, batch.edge_attr.view(-1, 1))


Client 1 Validation Loss: 13.479087829589844
Client 1 Gradients: {'conv1.bias': -2.9728012084960938, 'conv1.lin.weight': -0.0005810576840303838, 'conv2.bias': -0.5287426710128784, 'conv2.lin.weight': -0.08147692680358887, 'fc.weight': -0.9975160360336304, 'fc.bias': -7.319465637207031}
Client Epoch 1, Loss: 13.479087829589844
Client 2 Validation Loss: 11.068203926086426
Client 2 Gradients: {'conv1.bias': -3.824488878250122, 'conv1.lin.weight': -0.0007477003964595497, 'conv2.bias': -0.6182824373245239, 'conv2.lin.weight': -0.13989229500293732, 'fc.weight': -1.3615922927856445, 'fc.bias': -6.718288898468018}
Client Epoch 1, Loss: 11.068203926086426
Client 3 Validation Loss: 8.517389297485352
Client 3 Gradients: {'conv1.bias': -4.155227184295654, 'conv1.lin.weight': -0.0008128521149046719, 'conv2.bias': -0.6055759191513062, 'conv2.lin.weight': -0.15982277691364288, 'fc.weight': -1.6808114051818848, 'fc.bias': -5.870171546936035}
Round 2 Global Validation Loss: 8.517389297485352
Server Gra